In [8]:
import requests
import requests_cache
requests_cache.install_cache('cache')
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
import json
from pprint import pprint
API_KEY = 'pk.eyJ1IjoibnlvdTA0NSIsImEiOiJjaXJ6bDYxeWcwMXRzMnlwa2tmbzduNmd0In0.gLrTUpqhS3PYYy_AtCp4Gw'
BASE_URL = f"https://api.mapbox.com/search/searchbox/v1/category/services?access_token={API_KEY}&language=en&limit=25"

In [9]:
mb = gpd.read_file("statsnz-meshblock-2023-clipped-generalised-GPKG.zip!meshblock-2023-clipped-generalised.gpkg")
mb

,MB2023_V1_00,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,Shape_Length,geometry
0,0000100,12,Mainland,157.497825,157.497825,84526.158926,"MULTIPOLYGON (((172.67990 -34.42180, 172.67991..."
1,0000200,12,Mainland,120.503770,120.503770,122885.186789,"MULTIPOLYGON (((173.01433 -34.39285, 173.01461..."
2,0000300,12,Mainland,7.481859,7.481859,28321.227739,"MULTIPOLYGON (((172.87839 -34.49475, 172.88016..."
3,0000400,12,Mainland,83.342952,83.342952,41499.302475,"MULTIPOLYGON (((172.80881 -34.50158, 172.81036..."
4,0001201,12,Mainland,51.238250,51.238250,62944.884996,"MULTIPOLYGON (((173.06066 -34.70403, 173.06077..."
...,...,...,...,...,...,...,...
56783,4019154,12,Mainland,0.036001,0.036001,1105.893029,"MULTIPOLYGON (((174.80358 -41.25055, 174.80370..."
56784,4019155,12,Mainland,0.019734,0.019734,661.167326,"MULTIPOLYGON (((174.80411 -41.25295, 174.80383..."
56785,4019156,12,Mainland,0.097215,0.097215,1787.390321,"MULTIPOLYGON (((174.79657 -41.25581, 174.79724..."
56786,4019157,12,Mainland,0.066195,0.066195,1083.336705,"MULTIPOLYGON (((174.78952 -41.26050, 174.79035..."


In [10]:
mb["retrieved_at"] = pd.NA
mb["n_POIs"] = pd.NA
mb["bounds"] = mb.bounds.apply(lambda b: f"{b.minx},{b.miny},{b.maxx},{b.maxy}", axis=1)
mb["centroid"] = mb.centroid.apply(lambda p: f"{p.x},{p.y}")
mb.drop(columns="geometry", inplace=True)
mb.to_csv("locations.csv", index=False)

/tmp/ipykernel_2639511/1216559607.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mb["centroid"] = mb.centroid.apply(lambda p: f"{p.x},{p.y}")


In [11]:
results = {}
with open("data.geojson") as f:
    geojson = json.load(f)
    for feature in geojson["features"]:
        results[feature["properties"]["mapbox_id"]] = feature
len(results)

23401

In [17]:
for i, row in tqdm(mb.iterrows(), total=len(mb)):
    try:
        geojson = requests.get(BASE_URL, params={"proximity": row.centroid, "bbox": row.bounds}).json()
    except Exception as e:
        print(e)
        break
    for feature in geojson["features"]:
        results[feature["properties"]["mapbox_id"]] = feature
    mb.retrieved_at[i] = pd.Timestamp.now()
    mb.n_POIs[i] = len(geojson["features"])
    if i % 1000 == 0 or i == mb.index.max():
        mb.to_csv("locations.csv", index=False)
        geojson = {"type": "FeatureCollection", "features": list(results.values())}
        with open("data.geojson", "w") as f:
            json.dump(geojson, f)
mb

  0%|          | 0/56788 [00:00<?, ?it/s]

/tmp/ipykernel_2639511/2715605100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb.retrieved_at[i] = pd.Timestamp.now()
/tmp/ipykernel_2639511/2715605100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb.n_POIs[i] = len(geojson["features"])


,MB2023_V1_00,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,Shape_Length,retrieved_at,n_POIs,bounds,centroid
0,0000100,12,Mainland,157.497825,157.497825,84526.158926,2023-08-24 11:37:18.041099,0,"172.6352435839961,-34.54863701565286,172.87829...","172.76733837985918,-34.478922829717746"
1,0000200,12,Mainland,120.503770,120.503770,122885.186789,2023-08-24 11:37:20.533876,0,"172.8542824170829,-34.52161633328387,173.05212...","172.94698170393258,-34.45655876543814"
2,0000300,12,Mainland,7.481859,7.481859,28321.227739,2023-08-24 11:37:20.538599,0,"172.84206983288533,-34.53278549976881,172.9225...","172.87976545025646,-34.50977846981489"
3,0000400,12,Mainland,83.342952,83.342952,41499.302475,2023-08-24 11:37:20.541395,0,"172.75759288462214,-34.6271183160912,172.89445...","172.8250443171708,-34.56058525557085"
4,0001201,12,Mainland,51.238250,51.238250,62944.884996,2023-08-24 11:37:20.544155,1,"173.033969949182,-34.82551108204762,173.174736...","173.10101978845853,-34.75634587362149"
...,...,...,...,...,...,...,...,...,...,...
56783,4019154,12,Mainland,0.036001,0.036001,1105.893029,2023-08-24 18:08:22.089881,0,"174.80134530826163,-41.25386461779594,174.8048...","174.8032958014029,-41.252384905093116"
56784,4019155,12,Mainland,0.019734,0.019734,661.167326,2023-08-24 18:08:22.473459,0,"174.80181702413108,-41.25488188310054,174.8042...","174.80300326734843,-41.25396061008679"
56785,4019156,12,Mainland,0.097215,0.097215,1787.390321,2023-08-24 18:08:22.930917,6,"174.79049541680982,-41.26110144968412,174.7972...","174.79374495848225,-41.25827750871569"
56786,4019157,12,Mainland,0.066195,0.066195,1083.336705,2023-08-24 18:08:23.313995,4,"174.78636471703436,-41.2633821068233,174.79103...","174.78903872191805,-41.262061277837624"


In [ ]:
pd.Series([r["properties"]["external_ids"].keys() for r in results.values()]).value_counts()

(foursquare, safegraph)    3
(foursquare)               2
dtype: int64